In [23]:
# python modules we use
import os
import sys
from matplotlib import pyplot as plt

In [24]:
# setting up the environment
shyft_path = os.path.abspath(os.path.realpath('../hycamp/project_shyft/shyft'))
sys.path.append(shyft_path)

In [25]:
 from shyft.orchestration2.shyft_runner import *

In [26]:
# set up configuration
config_file = "./atnsjoen_catchment/calibration.yaml"
config_section = "atnsjoen"
config = CalibrationConfig(config_file, config_section)


In [27]:
# set up the simulator
calibrator = Calibrator(config)

In [28]:
# create a time axis
time_axis = api.Timeaxis(config.model_config.start_time, config.model_config.run_time_step, config.model_config.number_of_steps) 

In [29]:
# Initialize the calibrator
calibrator.init(time_axis)
calibrator.calibrator.set_verbose_level(1)

5 wind_speed series found.
1 radiation series found.
2 precipitation series found.
2 temperature series found.
1 relative_humidity series found.
Done with model setup/interpolation step
atnsjoen discharge [0]
Calibrator catchment index = [{'internal_id': 'atnsjoen', 'catch_id': [1]}]


In [56]:
# RUN THE CALIBRATION!
calibr_results = calibrator.calibrate(tol=1.0e-5)

Calibrating...


In [55]:
for key in calibr_results.keys():
    print '{:30}{}'.format(key, calibr_results[key])

wind_const                    1.0
max_albedo                    0.999869505569
p_corr_scale_factor           1.0626850716
fast_albedo_decay_rate        11.559230898
TX                            0.460644605192
glacier_albedo                0.4
surface_magnitude             30.0
snowfall_reset_depth          5.0
wind_scale                    0.361484623245
slow_albedo_decay_rate        35.068910805
ae_scale_factor               0.883940233255
c3                            -0.0576952850277
c2                            0.635572009024
c1                            -4.23683219445
snow_cv                       0.4
min_albedo                    0.407453875181
max_water                     0.1


In [ ]:
# save the calibration results
calibrator.save_calibrated_model(config.calibrated_model_file, calibr_results)

In [36]:
discharge = calibrator.target_ts['discharge']

In [47]:
# todo: get discharge time series; run a simulation; get simulation time series; define NSE function; compare and plot; Why is NSE low?; -> Compare only for actual calibration time. Get NSE for evaluation time! -> read discharge from  source file!!